### Automate tickets remainder process

The main goal of this python code is to automate the process of sending emails to employees that have tickets open for more than 3 days. 
This task has been performed with excel VBA but python can do the exact same thing in 3 minutes. In the past it would take me about 3 hours per week to send out the remainder emails to everyone on the team.

In [1]:
# import library needed

import pandas as pd
import smtplib
import datetime
import os
import re

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

In [2]:
# open document
data = pd.read_excel('Fin & HR - Updated more than 3 days ago.xlsx')
data.head()

,Number,Description,Additional comments,Opened,State,Assigned to,Assignment group,Short description,Updated,Updated by,Opened by,Watch list
0,FR06084412,NaN,"2019-04-22 17:20:17 CDT - Toni\nHello,\n\nThan...",2019-04-22 15:08:00,Work in Progress,Daniela Matinho (danielamatinho),sso--p2p-vendor,Vendor Set-up or Modify - NCOMPASS,2019-10-11 12:16:19,jcurreri,Lauren Bautista (lbautista1),NaN
1,FR06084410,NaN,2019-04-22 17:20:17 CDT - Tonirenee Arnold (Ad...,2019-04-22 15:08:05,Work in Progress,Joseph Curreri (jcurreri),sso--p2p-vendor,Vendor Set-up or Modify - NCOMPASS ENTERPRISES...,2019-10-11 12:16:19,jcurreri,Lauren Bautista (lbautista1),NaN
2,FR06109591,NaN,2019-10-09 08:07:41 \nComments from Task FT060...,2019-10-08 12:54:50,Work in Progress,Davene Cornell (dcornell),sso--p2p-vendor,Vendor Set-up or Modify - Immucor GTI Diagnost...,2019-10-09 09:58:08,dcornell,Qiang Wang (qwang3),NaN
3,FR06087748,NaN,2019-06-05 13:31:17 CDT - Lauren Bautista (Add...,2019-06-05 13:31:13,Work in Progress,Joseph Curreri (jcurreri),sso-p2p-vendor-duplication-queries,Vendor Set-up or Modify - THE*KIDS' TABLE LLC ...,2019-10-11 12:12:56,jcurreri,Lauren Bautista (lbautista1),NaN
4,FR06108204,NaN,2019-10-03 13:44:48 CDT - Hillary Carroll (Add...,2019-10-03 13:44:45,Work in Progress,Kimberly Shannon (kimberlyshannon),sso-helpdesk-p2p,Ask a Purchasing/Payment Question - Hillary Ca...,2019-10-10 09:14:13,kimberlyshannon,Hillary Carroll (hillaryc),NaN


In [97]:
# dimension of data
data.shape

(208, 12)

In [3]:
# people assigned with tickets from sso
df = data[(data['Assignment group']=='sso-helpdesk-hr') | (data['Assignment group']=='sso-helpdesk-p2p') 
     | (data['Assignment group']=='sso-management')| (data['Assignment group']=='sso-p2p-ach-sua')
    | (data['Assignment group']=='sso-p2p-qc') | (data['Assignment group']=='sso-p2p-reconciliation')
    | (data['Assignment group']=='sso--p2p-vendor') | (data['Assignment group']=='sso-payroll-ic') 
    | (data['Assignment group']=='sso-payroll-other') | (data['Assignment group']=='sso-payroll-tax')
     | (data['Assignment group']=='sso-hr-recruitment-retention')| (data['Assignment group']=='sso-hr-student-employment')
    | (data['Assignment group']=='sso-p2p--payment-support-2nd-level')|(data['Assignment group']=='sso-p2p--invoices-2nd-level')
    | (data['Assignment group']=='sso-fas-updates')| (data['Assignment group']=='sso-helpdesk')
          | (data['Assignment group']=='sso-hr-data-management')| (data['Assignment group']=='sso-p2p-expense')
          | (data['Assignment group']=='sso-p2p-buysitecomments')| (data['Assignment group']=='sso-payroll-non-us')
          | (data['Assignment group']=='sso-payroll-other')]
df

,Number,Description,Additional comments,Opened,State,Assigned to,Assignment group,Short description,Updated,Updated by,Opened by,Watch list
0,FR06084412,NaN,"2019-04-22 17:20:17 CDT - Toni\nHello,\n\nThan...",2019-04-22 15:08:00,Work in Progress,Daniela Matinho (danielamatinho),sso--p2p-vendor,Vendor Set-up or Modify - NCOMPASS,2019-10-11 12:16:19,jcurreri,Lauren Bautista (lbautista1),NaN
1,FR06084410,NaN,2019-04-22 17:20:17 CDT - Tonirenee Arnold (Ad...,2019-04-22 15:08:05,Work in Progress,Joseph Curreri (jcurreri),sso--p2p-vendor,Vendor Set-up or Modify - NCOMPASS ENTERPRISES...,2019-10-11 12:16:19,jcurreri,Lauren Bautista (lbautista1),NaN
2,FR06109591,NaN,2019-10-09 08:07:41 \nComments from Task FT060...,2019-10-08 12:54:50,Work in Progress,Davene Cornell (dcornell),sso--p2p-vendor,Vendor Set-up or Modify - Immucor GTI Diagnost...,2019-10-09 09:58:08,dcornell,Qiang Wang (qwang3),NaN
4,FR06108204,NaN,2019-10-03 13:44:48 CDT - Hillary Carroll (Add...,2019-10-03 13:44:45,Work in Progress,Kimberly Shannon (kimberlyshannon),sso-helpdesk-p2p,Ask a Purchasing/Payment Question - Hillary Ca...,2019-10-10 09:14:13,kimberlyshannon,Hillary Carroll (hillaryc),NaN
5,FR06082857,Email To: pps@uchicago.edu\n\nEmail From: bwil...,2019-04-10 16:55:57 CDT - Bonnie Perryman (Add...,2019-04-02 09:32:37,Work in Progress,Joseph Curreri (jcurreri),sso-p2p-ach-sua,NoIR Laser Company Supplier Application Form,2019-04-19 11:24:35,bbyham,Guest User (Guest),"jhodges@noirlaser.com, noiraccounting@noirlase..."
6,FR06082203,Email To: vendorsupport@uchicago.edu\n\nEmail ...,NaN,2019-03-22 14:40:56,Work in Progress,Joseph Curreri (jcurreri),sso-p2p-ach-sua,CGX LLV Invoice Information re: PO #G481389 / ...,2019-03-26 09:54:19,jcurreri,Guest User (Guest),NaN
7,FR06110657,NaN,2019-10-10 15:06:49 CDT - Traci Matthews (Addi...,2019-10-10 14:49:56,Work in Progress,Tonirenee Arnold (tonireneej),sso-p2p-expense,Submit Expenses/Reimbursements - North Carolin...,2019-10-11 16:37:02,traci1,Katherine Hamaguchi (khamaguchi),NaN
9,FR06110928,NaN,2019-10-11 12:11:35 CDT - Mary Hansen (Additio...,2019-10-11 12:11:31,Work in Progress,Judy Gierat (jgierat),sso--p2p-vendor,Ask a Purchasing/Payment Question - Mary Hanse...,2019-10-11 15:08:31,jgierat,Mary Hansen (mhansen6),NaN
10,FR06107584,Email To: vendorsupport@uchicago.edu\n\nEmail ...,2019-10-04 08:22:58 CDT - Joann Campbell (Addi...,2019-10-02 09:49:43,Work in Progress,Joann Campbell (jcampbell),sso-payroll-ic,Fwd: O503789-Stacy Adams,2019-10-04 08:22:58,jcampbell,Ryan Priester (rkpriest),NaN
11,FR06092901,Email To: pps@uchicago.edu\n\nEmail From: egil...,NaN,2019-08-14 10:42:56,Work in Progress,Hector Garcia (hgarcia3),sso-helpdesk-p2p,Status of Check Request,2019-09-09 13:45:02,hgarcia3,Guest User (Guest),egilmore@bsd.uchicago.edu


In [65]:
df.shape

(174, 12)

In [99]:
# divide the fin tickets in personal files
for name, g in df.groupby('Assigned to'):
    g.to_excel(f'{name}.xlsx', index=False)

In [4]:
# get the path where all files arfe stored
path = 'Users/yourname/Documents/E-ticketing/20191015'
path = os.getcwd()
files = os.listdir(path)
files

['.ipynb_checkpoints',
 'Adelaide Swida (adelaide).xlsx',
 'Ashanti Bentley (abentley).xlsx',
 'Brittany Pickens (bpickens).xlsx',
 'Calvin Lewis (clewis).xlsx',
 'Chavon Coleman (chavonc).xlsx',
 'Claudia Bakken (cbakken).xlsx',
 'Daniela Matinho (danielamatinho).xlsx',
 'Davene Cornell (dcornell).xlsx',
 'Fin & HR - Updated more than 3 days ago.xlsx',
 'fin_10072019.ipynb',
 'fin_10202019.ipynb',
 'fin_10202019_github.ipynb',
 'Hector Garcia (hgarcia3).xlsx',
 'Joann Campbell (jcampbell).xlsx',
 'Joseph Curreri (jcurreri).xlsx',
 'Judy Gierat (jgierat).xlsx',
 'Karey Middleton (kareym).xlsx',
 'Kimberly Shannon (kimberlyshannon).xlsx',
 'Mary Mcclelland (mcclellandm).xlsx',
 'Maya Moseberry (mmoseberry).xlsx',
 'Sara Hayden (shayden).xlsx',
 'Shakita Moore-Bennett (shakita1).xlsx',
 'Sharmeika Phillips (sharphillips).xlsx',
 'Tonirenee Arnold (tonireneej).xlsx',
 'Traci Matthews (traci1).xlsx']

In [102]:
# selected xlsx as they correspond to ticket's files
files_xlsx = [f for f in files if f[-6:] == ').xlsx']
print(files_xlsx)
type(files_xlsx)

['Adelaide Swida (adelaide).xlsx', 'Ashanti Bentley (abentley).xlsx', 'Brittany Pickens (bpickens).xlsx', 'Calvin Lewis (clewis).xlsx', 'Chavon Coleman (chavonc).xlsx', 'Claudia Bakken (cbakken).xlsx', 'Daniela Matinho (danielamatinho).xlsx', 'Davene Cornell (dcornell).xlsx', 'Hector Garcia (hgarcia3).xlsx', 'Joann Campbell (jcampbell).xlsx', 'Joseph Curreri (jcurreri).xlsx', 'Judy Gierat (jgierat).xlsx', 'Karey Middleton (kareym).xlsx', 'Kimberly Shannon (kimberlyshannon).xlsx', 'Mary Mcclelland (mcclellandm).xlsx', 'Maya Moseberry (mmoseberry).xlsx', 'Sara Hayden (shayden).xlsx', 'Shakita Moore-Bennett (shakita1).xlsx', 'Sharmeika Phillips (sharphillips).xlsx', 'Tonirenee Arnold (tonireneej).xlsx', 'Traci Matthews (traci1).xlsx']


list

In [103]:
# get list of names
str1 = ''.join(files_xlsx)
index = re.findall(r'\((.*?)\)', str1)
print(index) 

# get list of names and add university email
string = '@uchicago.edu'
emails_list = [x + string for x in index]
print (emails_list)

['adelaide', 'abentley', 'bpickens', 'clewis', 'chavonc', 'cbakken', 'danielamatinho', 'dcornell', 'hgarcia3', 'jcampbell', 'jcurreri', 'jgierat', 'kareym', 'kimberlyshannon', 'mcclellandm', 'mmoseberry', 'shayden', 'shakita1', 'sharphillips', 'tonireneej', 'traci1']
['adelaide@uchicago.edu', 'abentley@uchicago.edu', 'bpickens@uchicago.edu', 'clewis@uchicago.edu', 'chavonc@uchicago.edu', 'cbakken@uchicago.edu', 'danielamatinho@uchicago.edu', 'dcornell@uchicago.edu', 'hgarcia3@uchicago.edu', 'jcampbell@uchicago.edu', 'jcurreri@uchicago.edu', 'jgierat@uchicago.edu', 'kareym@uchicago.edu', 'kimberlyshannon@uchicago.edu', 'mcclellandm@uchicago.edu', 'mmoseberry@uchicago.edu', 'shayden@uchicago.edu', 'shakita1@uchicago.edu', 'sharphillips@uchicago.edu', 'tonireneej@uchicago.edu', 'traci1@uchicago.edu']


In [104]:
# dictionary with managers and employees that will recieve email with tickets open
# people - managers
managers = {'teammember@uchicago.edu':'manager@uchicago.edu'            
           }

In [109]:
def send_email_work(f):
    user = re.findall(r'\((.*?)\)', f)[0]
    end_email = '@uchicago.edu'
    single_email = user + end_email
    print(single_email)    
    
    cc_email = []
    if single_email in managers:
        cc_email.append(managers[single_email])

    # email details 
    email_user = 'danielamatinho@uchicago.edu'
    email_password = 'Aus18$UCG19%'
    email_send = single_email
    subject = 'ServicesNow ticekts'
    cc = cc_email
    
    # compose msg
    msg = MIMEMultipart()
    msg['From'] = email_user
    msg['To'] = email_send
    msg['Subject'] = subject
    msg['CC'] = ','.join(cc)
    date = datetime.date.today().strftime('%m-%d-%Y')


    # msg with attachment file
    body = 'Hello, \n\nThis is courtesy email regarding ServiceNow tickets. Attached are SN tickets assigned to you that, as of '+ date + ', have not been updated in three or more days.'
    msg.attach(MIMEText(body, 'plain'))
    filename = f
    attachment = open(filename, 'rb')

    # set a part to read the email and attachated doc
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+filename)
    msg.attach(part)

    # text of the email
    text = msg.as_string()

    # connect to server
    email_send = [single_email] + cc
    #email_send = email_send
    
    server = smtplib.SMTP('smtp-mail.outlook.com', 587)
    server.starttls()
    server.login(email_user,email_password)

    # send and close application
    server.sendmail(email_user,email_send,text)
    server.quit()
    return


In [111]:
for f in files:
    if f[-6:] == ').xlsx': 
        if f[0].isupper():
            print(f)
            send_email_work(f)
            

Adelaide Swida (adelaide).xlsx
adelaide@uchicago.edu
Ashanti Bentley (abentley).xlsx
abentley@uchicago.edu
Brittany Pickens (bpickens).xlsx
bpickens@uchicago.edu
Calvin Lewis (clewis).xlsx
clewis@uchicago.edu
Chavon Coleman (chavonc).xlsx
chavonc@uchicago.edu
Claudia Bakken (cbakken).xlsx
cbakken@uchicago.edu
Daniela Matinho (danielamatinho).xlsx
danielamatinho@uchicago.edu
Davene Cornell (dcornell).xlsx
dcornell@uchicago.edu
Hector Garcia (hgarcia3).xlsx
hgarcia3@uchicago.edu
Joann Campbell (jcampbell).xlsx
jcampbell@uchicago.edu
Joseph Curreri (jcurreri).xlsx
jcurreri@uchicago.edu
Judy Gierat (jgierat).xlsx
jgierat@uchicago.edu
Karey Middleton (kareym).xlsx
kareym@uchicago.edu
Kimberly Shannon (kimberlyshannon).xlsx
kimberlyshannon@uchicago.edu
Mary Mcclelland (mcclellandm).xlsx
mcclellandm@uchicago.edu
Maya Moseberry (mmoseberry).xlsx
mmoseberry@uchicago.edu
Sara Hayden (shayden).xlsx
shayden@uchicago.edu
Shakita Moore-Bennett (shakita1).xlsx
shakita1@uchicago.edu
Sharmeika Phillips